<a href="https://colab.research.google.com/github/prodbrown/SWIPE-TOOL/blob/main/TV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# ✅ Install Gradio if needed
try:
    import gradio as gr
except ImportError:
    !pip install gradio
    import gradio as gr

import requests

# ✅ Global cache
channels_data = []
streams_data = []
country_channel_map = {}

# ✅ Load and index TV data from iptv-org
def load_iptv_data():
    global channels_data, streams_data, country_channel_map
    print("📡 Fetching IPTV data...")

    # Channels: metadata like name, country, etc.
    channels_url = "https://iptv-org.github.io/api/channels.json"
    streams_url = "https://iptv-org.github.io/api/streams.json"

    channels_data = requests.get(channels_url).json()
    streams_data = requests.get(streams_url).json()

    # Map: country -> list of (channel_name, stream_url)
    country_channel_map = {}

    # Build stream lookup table: channel_id -> .m3u8 stream
    stream_lookup = {}
    for stream in streams_data:
        if stream.get("channel") and stream.get("url") and stream["url"].endswith(".m3u8"):
            stream_lookup.setdefault(stream["channel"], []).append(stream["url"])

    # Build map
    for ch in channels_data:
        country = ch.get("country")
        name = ch.get("name")
        channel_id = ch.get("id")
        streams = stream_lookup.get(channel_id)
        if country and name and streams:
            country_channel_map.setdefault(country, []).append((name, streams[0]))  # take first stream

    return sorted(country_channel_map.keys())

# ✅ Load channels when country selected
def get_channels_by_country(country):
    channels = country_channel_map.get(country, [])
    names = [c[0] for c in channels]
    return names, {c[0]: c[1] for c in channels}

# ✅ Play selected stream
def play_tv_channel(channel_name, stream_dict):
    stream_url = stream_dict.get(channel_name)
    if not stream_url:
        return "❌ No stream found."
    return f"""
    <video width="640" height="360" controls autoplay>
        <source src="{stream_url}" type="application/x-mpegURL">
        Your browser does not support the video tag.
    </video>
    """

# ✅ Build Gradio App
def build_app():
    country_list = load_iptv_data()

    with gr.Blocks() as demo:
        gr.Markdown("# 📺 **Global TV Stream Player**\nChoose a country and watch live TV online.")

        country_dropdown = gr.Dropdown(label="🌍 Country", choices=country_list, interactive=True)
        load_btn = gr.Button("🔄 Load TV Channels")

        channel_dropdown = gr.Dropdown(label="📡 Available Channels", choices=[], interactive=True)
        status_text = gr.Textbox(label="ℹ️ Status", interactive=False)

        play_btn = gr.Button("▶️ Play")
        video_output = gr.HTML()

        # Store channel name → stream_url mapping
        stream_dict_state = gr.State({})

        # Load channels for selected country
        def load_channels(country):
            channels, stream_map = get_channels_by_country(country)
            if not channels:
                return gr.update(choices=[]), "❌ No channels found.", {}
            return gr.update(choices=channels, value=channels[0]), f"✅ {len(channels)} channels found in {country}.", stream_map

        load_btn.click(fn=load_channels,
                       inputs=country_dropdown,
                       outputs=[channel_dropdown, status_text, stream_dict_state])

        play_btn.click(fn=play_tv_channel,
                       inputs=[channel_dropdown, stream_dict_state],
                       outputs=video_output)

    demo.launch()

# ✅ Run it
build_app()

📡 Fetching IPTV data...
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://04183cc622b5837be4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
